In [0]:
'''!pip install tensorflow-gpu==2.0
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece'''

'!pip install tensorflow-gpu==2.0\n!pip install tensorflow_hub\n!pip install bert-for-tf2\n!pip install sentencepiece'

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.0.0
Hub version:  0.8.0


In [0]:
import bert
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
#import math
import numpy as np
import zipfile
import json
import os
import glob
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
basePath = '/content/'

data_dir = basePath+'dataset'
drive_dir = basePath+'drive/My Drive/temp/DatasetN'
paraFile = basePath + 'drive/My Drive/nlpproject/paragraph2.json'

'''data_dir = basePath+'tempdataset'
drive_dir = basePath+'drive/My Drive/temp/tempDataset'
paraFile = basePath + 'drive/My Drive/temp.json' '''

outDir = basePath+'test'

max_seq_length = 512

In [0]:
'''for file_name in os.listdir(drive_dir):
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:
            zip_dir.extractall(path='/content/')'''

"for file_name in os.listdir(drive_dir):\n    if file_name.endswith('.zip'):\n        with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:\n            zip_dir.extractall(path='/content/')"

In [0]:
def getBERTModel():
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="segment_ids")

    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)
    #bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1",trainable=False)
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    FullTokenizer = bert.bert_tokenization.FullTokenizer
    tokenizer = FullTokenizer(vocab_file, do_lower_case)

    return {'model':model,'tokenizer':tokenizer}

In [0]:
def get_masks(tokens, max_seq_length):
    #print('len(tokens),max_seq_length)
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [0]:
def getEmbeddings(model,tokenizer,sentence): 
    stokens = tokenizer.tokenize(sentence)

    if len(stokens) > (max_seq_length - 2):
      stokens = stokens[:max_seq_length-2]

    stokens = ["[CLS]"] + stokens + ["[SEP]"]

    input_ids = get_ids(stokens, tokenizer, max_seq_length)
    #print(len(input_ids))
    input_masks = get_masks(stokens, max_seq_length)
    input_segments = get_segments(stokens, max_seq_length)

    '''print(input_masks)
    print(input_segments)'''

    pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
    '''print('see')
    print(all_embs.shape)
    print(pool_embs.shape)'''
    # pool_ebmbs is an embeding of CLS token
    # all_embs contains embeding for words of input sentence.
    return pool_embs

In [0]:
def BERTEmbeddings(model,tokenizer,nSentences):
    trainX = np.asarray([])
    n = len(nSentences)
    for i in range(n): 
        if i%500 == 0:
          print( 'Processing ',i,' out of ',n)

        #senLen = len(nSentences[i].split())
        embs = getEmbeddings(model,tokenizer,nSentences[i])
        '''print(embs.shape)
        x = embs'''
        if trainX.shape[0] == 0:
            trainX = embs
        else:
            trainX = np.concatenate((trainX, embs), axis=0)
    return trainX

In [0]:
def getFileSent(inpFile):
    fileSent = ''
    article = ''
    with open(inpFile,'r') as f:
        x = json.loads(f.read())
        article = x['article']
        for option in x['options'][0]:
            fileSent += option
            fileSent += ' '
        fileSent += x['questions'][0]
    return fileSent,article

In [0]:
def saveFile(inpFile,outFile,para):
    with open(inpFile,'r') as f:
      x = json.loads(f.read())
      x['article'] = para

      with open(outFile, 'w') as jsonOut:
          json.dump(x, jsonOut)

In [0]:
outModel = getBERTModel()
model = outModel['model']
tokenizer = outModel['tokenizer']

In [0]:
'''f = open(paraFile,'r')
x = json.loads(f.read())
f.close()
fileSents = [ x['para'+str(id)] for id in range(1,len(x)+1)]'''
#print(len(fileSents))

"f = open(paraFile,'r')\nx = json.loads(f.read())\nf.close()\nfileSents = [ x['para'+str(id)] for id in range(1,len(x)+1)]"

In [0]:
'''paraEembs = BERTEmbeddings(model,tokenizer,fileSents)
print(paraEembs.shape)'''

'paraEembs = BERTEmbeddings(model,tokenizer,fileSents)\nprint(paraEembs.shape)'

In [0]:
#np.savetxt(basePath+'paraEmbs.csv',paraEembs, delimiter=',',fmt='%.15f')

In [0]:
#totalfile = sum([len(files) for r, d, files in os.walk("/content/dataset/test/")])
totalfile = sum([len(files) for r, d, files in os.walk(data_dir+"/train/")])
#totalfile = sum([len(files) for r, d, files in os.walk("/content/dataset/dev/")])


print(totalfile)
count = 0
setData = []
for i in range(1,totalfile+1):
  path = data_dir + "/train/" + "traindata" +str(i)+".json"
  f = open(path,"r")
  testdata = json.loads(f.read())
  f.close()
  
  arti = testdata['article']
  que = testdata['questions'][0]
  setData.append(arti)
  setData.append(que)

setDataEmb = BERTEmbeddings(model,tokenizer,setData)
print(setDataEmb.shape)
artEmb = setDataEmb[::2]
qusEmb = setDataEmb[1::2]
print(artEmb.shape)
print(qusEmb.shape)
saveData = np.concatenate((artEmb,artEmb), axis=1)
print(saveData.shape)
np.savetxt(basePath+'trainEmbs.csv',saveData, delimiter=',',fmt='%.15f')


'''setDataEmb = list(setDataEmb)
print(len(setDataEmb),len(setDataEmb[0]),count)
l_set = len(setDataEmb)
newData = []
for i in range(0,l_set,2):
  newData.append(np.concatenate([setDataEmb[i],setDataEmb[i+1]]))

print(len(newData),len(newData[0]))
#np.savetxt(basePath+'testDataEmbs.csv',newData, delimiter=',',fmt='%.15f')
np.savetxt(basePath+'trainDataEmbs.csv',newData, delimiter=',',fmt='%.15f')
#np.savetxt(basePath+'validDataEmbs.csv',newData, delimiter=',',fmt='%.15f')'''

10481
Processing  0  out of  20962
Processing  500  out of  20962
Processing  1000  out of  20962
Processing  1500  out of  20962
Processing  2000  out of  20962
Processing  2500  out of  20962
Processing  3000  out of  20962
Processing  3500  out of  20962
Processing  4000  out of  20962
Processing  4500  out of  20962
Processing  5000  out of  20962
Processing  5500  out of  20962
Processing  6000  out of  20962
Processing  6500  out of  20962
Processing  7000  out of  20962
Processing  7500  out of  20962
Processing  8000  out of  20962
Processing  8500  out of  20962
Processing  9000  out of  20962
Processing  9500  out of  20962
Processing  10000  out of  20962
Processing  10500  out of  20962
Processing  11000  out of  20962
Processing  11500  out of  20962
Processing  12000  out of  20962
Processing  12500  out of  20962
Processing  13000  out of  20962
Processing  13500  out of  20962
Processing  14000  out of  20962
Processing  14500  out of  20962
Processing  15000  out of  2

"setDataEmb = list(setDataEmb)\nprint(len(setDataEmb),len(setDataEmb[0]),count)\nl_set = len(setDataEmb)\nnewData = []\nfor i in range(0,l_set,2):\n  newData.append(np.concatenate([setDataEmb[i],setDataEmb[i+1]]))\n\nprint(len(newData),len(newData[0]))\n#np.savetxt(basePath+'testDataEmbs.csv',newData, delimiter=',',fmt='%.15f')\nnp.savetxt(basePath+'trainDataEmbs.csv',newData, delimiter=',',fmt='%.15f')\n#np.savetxt(basePath+'validDataEmbs.csv',newData, delimiter=',',fmt='%.15f')"

In [0]:
np.savetxt(basePath+'drive/My Drive/temp/trainEmbs.csv',saveData, delimiter=',',fmt='%.15f')

In [0]:
#print("temp")

In [0]:
#print("temp")

In [0]:
#print("temp")